In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [2]:
%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper
from ml.models.base.OptunaRay.brisk_bagging import BriskBagging

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common
import ray
import time
import pandas as pd

In [4]:
rayer.get_global_cluster(num_cpus=4)

(scheduler +17s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +17s) Removing 7 nodes of type small-group (idle).
(scheduler +28s) Resized to 11 CPUs.
(scheduler +35s) Removing 1 nodes of type small-group (idle).
(scheduler +42s) Resized to 13 CPUs.
(scheduler +42s) Removing 2 nodes of type small-group (idle).
(scheduler +50s) Resized to 17 CPUs.
(scheduler +58s) Resized to 19 CPUs.
(scheduler +1m13s) Resized to 21 CPUs.
(scheduler +1m43s) Removing 2 nodes of type small-group (idle).
(scheduler +1m51s) Resized to 23 CPUs.
(scheduler +1m51s) Removing 4 nodes of type small-group (idle).
(scheduler +1m58s) Resized to 25 CPUs.
(scheduler +1m58s) Removing 3 nodes of type small-group (idle).
(scheduler +2m14s) Resized to 21 CPUs.
(scheduler +2m14s) Removing 1 nodes of type small-group (idle).
(scheduler +2m22s) Resized to 23 CPUs.
(scheduler +2m30s) Resized to 25 CPUs.
(scheduler +2m53s) Removing 1 nodes of type small

(raylet, ip=10.10.13.61) *** SIGSEGV received at time=1669314570 on cpu 0 ***
(raylet, ip=10.10.13.61) PC: @     0x55d9429edbcf  (unknown)  boost::asio::detail::epoll_reactor::start_op()
(raylet, ip=10.10.13.61)     @     0x7f51ec0d8420       3440  (unknown)
(raylet, ip=10.10.13.61)     @     0x55d9429ef5b3         64  boost::asio::detail::reactive_socket_service_base::start_accept_op()
(raylet, ip=10.10.13.61)     @     0x55d942345d60        112  plasma::PlasmaStore::DoAccept()
(raylet, ip=10.10.13.61)     @     0x55d942345aca        368  boost::asio::detail::reactive_socket_accept_op<>::do_complete()
(raylet, ip=10.10.13.61)     @     0x55d9429f1bcb        128  boost::asio::detail::scheduler::do_run_one()
(raylet, ip=10.10.13.61)     @     0x55d9429f3391        192  boost::asio::detail::scheduler::run()
(raylet, ip=10.10.13.61)     @     0x55d9429f35c0         64  boost::asio::io_context::run()
(raylet, ip=10.10.13.61)     @     0x55d942342218        384  plasma::PlasmaStoreRunner::S

(scheduler +18m46s) Resized to 27 CPUs.
(scheduler +18m46s) Removing 2 nodes of type small-group (idle).
(scheduler +18m54s) Removing 2 nodes of type small-group (idle).
(scheduler +19m1s) Removing 2 nodes of type small-group (idle).
(scheduler +19m9s) Resized to 25 CPUs.
(scheduler +19m17s) Resized to 23 CPUs.
(scheduler +19m26s) Resized to 25 CPUs.
(scheduler +19m26s) Removing 1 nodes of type small-group (idle).
(scheduler +19m34s) Resized to 27 CPUs.
(scheduler +19m34s) Removing 1 nodes of type small-group (idle).
(scheduler +19m41s) Removing 1 nodes of type small-group (idle).
(scheduler +19m49s) Removing 1 nodes of type small-group (idle).
(scheduler +19m57s) Removing 1 nodes of type small-group (idle).
(scheduler +20m5s) Removing 2 nodes of type small-group (idle).
(scheduler +20m12s) Resized to 29 CPUs.
(scheduler +20m12s) Removing 1 nodes of type small-group (idle).
(scheduler +20m20s) Resized to 25 CPUs.
(scheduler +20m20s) Removing 1 nodes of type small-group (idle).
(schedul

2022-11-24 12:48:11,256	WARNING dataclient.py:396 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-11-24 12:48:20,877	ERROR dataclient.py:323 -- Unrecoverable error in data channel.
Unexpected exception:
Traceback (most recent call last):
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/ray/util/client/logsclient.py", line 67, in _log_main
    for record in log_stream:
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/grpc/_channel.py", line 426, in __next__
    return self._next()
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/grpc/_channel.py", line 826, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_Multi

In [ ]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

In [ ]:
ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
ds_train = common.label_encode(ds_train)
ds_test = common.label_encode(ds_test)

ds_train = ds_train.fillna(-1)
ds_test = ds_test.fillna(-1)

df_X = ds_train.loc[:, ds_train.columns != 'target']
df_y = ds_train['target']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)


ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)




In [ ]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)

In [ ]:
X_train_id = ray.put(X_train_scalar.values)
y_train_id = ray.put(y_train)
X_test_id = ray.put(X_test_scalar.values)
y_test_id = ray.put(y_test)

In [ ]:

@ray.remote(num_returns=1)
def worker(base_model, X_train, X_test, y_train, y_test):
    base_model.fetch_model(X_train, X_test, y_train, y_test)
    return base_model


In [ ]:
brisk_bag_1 = BriskBagging('brisk_xgb1')
brisk_bag_1.n_estimators = 10
brisk_bag_1.n_trials = 5


brisk_bag_2 = BriskBagging('brisk_xgb2')
brisk_bag_2.n_estimators = 10
brisk_bag_2.n_trials = 5

list_models = [brisk_bag_1, brisk_bag_2]

In [ ]:
#brisk_bag_1.fetch_model(X_train, X_test, y_train, y_test)
model_results = ray.get([worker.remote(base_model, X_train_id, X_test_id, y_train_id, y_test_id) for base_model in list_models])

In [15]:
from ray import tune

# 1. Define an objective function.
def objective(config):
    score = config["a"] ** 2 + config["b"]
    return {"score": score}


# 2. Define a search space.
search_space = {
    "a": tune.grid_search([0.001, 0.01, 0.1, 1.0]),
    "b": tune.choice([1, 2, 3]),
}

# 3. Start a Tune run and print the best result.
tuner = tune.Tuner(objective, param_space=search_space)
results = tuner.fit()
print(results.get_best_result(metric="score", mode="min").config)

(TunerInternal pid=363) 2022-11-24 07:25:40,942	WARNING function_trainable.py:586 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


(TunerInternal pid=363) == Status ==
(TunerInternal pid=363) Current time: 2022-11-24 07:25:46 (running for 00:00:02.08)
(TunerInternal pid=363) Memory usage on this node: 2.7/15.4 GiB 
(TunerInternal pid=363) Using FIFO scheduling algorithm.
(TunerInternal pid=363) Resources requested: 1.0/5 CPUs, 0/0 GPUs, 0.0/16.76 GiB heap, 0.0/4.91 GiB objects
(TunerInternal pid=363) Result logdir: /home/ray/ray_results/objective_2022-11-24_07-25-40
(TunerInternal pid=363) Number of trials: 4/4 (3 PENDING, 1 RUNNING)
(TunerInternal pid=363) +-----------------------+----------+-----------------+-------+-----+
(TunerInternal pid=363) | Trial name            | status   | loc             |     a |   b |
(TunerInternal pid=363) |-----------------------+----------+-----------------+-------+-----|
(TunerInternal pid=363) | objective_414eb_00000 | RUNNING  | 10.10.20.33:398 | 0.001 |   2 |
(TunerInternal pid=363) | objective_414eb_00001 | PENDING  |                 | 0.01  |   3 |
(TunerInternal pid=363) 

(TunerInternal pid=363) 2022-11-24 07:25:46,471	INFO tune.py:777 -- Total run time: 5.53 seconds (2.23 seconds for the tuning loop).
(worker pid=138, ip=10.10.63.163) /tmp/ray/session_2022-11-24_07-21-18_572881_8/runtime_resources/working_dir_files/_ray_pkg_e69997be12e85035/ml/models/base/OptunaRay/brisk_bagging.py:17: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
(worker pid=138, ip=10.10.63.163)   from ray.tune.suggest.optuna import OptunaSearch
(worker pid=138, ip=10.10.63.163) 2022-11-24T07:25:52PST : INFO : font_manager : _load_fontmanager : 1633 : Message : generated new fontManager
(worker pid=138, ip=10.10.63.163) /tmp/ray/session_2022-11-24_07-21-18_572881_8/runtime_resources/working_dir_files/_ray_pkg_e69997be12e85035/ml/models/base/OptunaRay/brisk_baggi

In [16]:
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch

space = {
    'width': (0, 20),
    'height': (-100, 100),
}
bayesopt = BayesOptSearch(space, metric="mean_loss", mode="min")
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        search_alg=bayesopt,
    ),
)
tuner.fit()

(TunerInternal pid=482) 2022-11-24 07:27:52,092	WARNING function_trainable.py:586 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


(TunerInternal pid=482) == Status ==
(TunerInternal pid=482) Current time: 2022-11-24 07:27:54 (running for 00:00:01.37)
(TunerInternal pid=482) Memory usage on this node: 2.8/15.4 GiB 
(TunerInternal pid=482) Using FIFO scheduling algorithm.
(TunerInternal pid=482) Resources requested: 1.0/5 CPUs, 0/0 GPUs, 0.0/16.76 GiB heap, 0.0/4.91 GiB objects
(TunerInternal pid=482) Result logdir: /home/ray/ray_results/objective_2022-11-24_07-27-52
(TunerInternal pid=482) Number of trials: 1/1 (1 RUNNING)
(TunerInternal pid=482) +--------------------+----------+------------------+----------+---------+
(TunerInternal pid=482) | Trial name         | status   | loc              |   height |   width |
(TunerInternal pid=482) |--------------------+----------+------------------+----------+---------|
(TunerInternal pid=482) | objective_901d919a | RUNNING  | 10.10.63.163:198 |  -25.092 | 19.0143 |
(TunerInternal pid=482) +--------------------+----------+------------------+----------+---------+
(TunerInte

(TunerInternal pid=482) 2022-11-24 07:27:54,535	ERROR trial_runner.py:993 -- Trial objective_901d919a: Error processing event.
(TunerInternal pid=482) ray.exceptions.RayTaskError(KeyError): ray::ImplicitFunc.train() (pid=198, ip=10.10.63.163, repr=objective)
(TunerInternal pid=482)   File "/home/ray/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/trainable.py", line 355, in train
(TunerInternal pid=482)     raise skipped from exception_cause(skipped)
(TunerInternal pid=482)   File "/home/ray/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 325, in entrypoint
(TunerInternal pid=482)     return self._trainable_func(
(TunerInternal pid=482)   File "/home/ray/anaconda3/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py", line 651, in _trainable_func
(TunerInternal pid=482)     output = fn()
(TunerInternal pid=482)   File "/tmp/ipykernel_20344/2397527074.py", line 5, in objective
(TunerInternal pid=482) KeyError: 'a'


(TunerInternal pid=482) 2022-11-24 07:27:54,643	ERROR tune.py:773 -- Trials did not complete: [objective_901d919a]
(TunerInternal pid=482) 2022-11-24 07:27:54,643	INFO tune.py:777 -- Total run time: 2.55 seconds (1.38 seconds for the tuning loop).
